 # Integrated Project 1

## Video Game Analysis

### Introduction

<div style="border-radius: 15px; border: 3px solid skyblue; padding: 15px;">
In this project, I will be demonstrating my skills on Exploratory and Statistical Data Analysis using a video game data set from 2016.

I will be preparing a campaign to forecast 2017 sales based on this data.

The goal will be to identify patterns that determine whether a game succeeds or not.

I plan to:
1. Read the data
2. Prepare the data (transform, fill missing values, remove duplicates, add necessary columns)
3. Perform Exploratory Data Analysis
4. Create a profile for each region in the dataset
5. Perform Statistical Data Analysis
6. Report on my findings

In [52]:
# import necessary libraries
import pandas as pd
import numpy as np 
from scipy import stats
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

<div style="border-radius: 15px; border: 3px solid skyblue; padding: 15px;">
Data Descriptions:

—Name 

—Platform 

—Year_of_Release 

—Genre 

—NA_sales (North American sales in USD million) 

—EU_sales (sales in Europe in USD million) 

—JP_sales (sales in Japan in USD million) 

—Other_sales (sales in other countries in USD million) 

—Critic_Score (maximum of 100) 

—User_Score (maximum of 10) 

—Rating (ESRB)

In [53]:
# read the data. Run on local, and if that fails, load on server side
try:
    df = pd.read_csv('games.csv')
except:
    df = pd.read_csv('/datasets/games.csv')

In [54]:
# take a look at the raw dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


#### <span style="color:skyblue">Initial Observations</span>
<div style="border-radius: 15px; border: 3px solid skyblue; padding: 15px;">

Column Names
 - The column names should all be lower case

Data Types
 - year_of_release should be int
 - user_score should be float


Missing Values
 - There are 2 missing values in the name and genre columns. This will need to be investigated.
 - There are quite a few missing year_of_release values, these will have to be assumed by the mean year_of_release date for that specific platform
 - critic and user scores have many missing values. These will have to be replaced by the mean for that platform.
 - There's many missing ratings too. This is likely because there were no ratings for games by ESRB prior to 1994. I will investigate if this is the case. It might be safe to replace all the missing values from after 1994 with E. For those prior to 1994, They can be replaced with a value like NA since they weren't rated if my theory is true.


 For the future:
 - Determine if a platform is dead by checking if the have any more released games in 2016. if not, exclude that platform from the analysis (should be 5 years)

### Prepare the data

In [55]:
# fix column names
df.columns = df.columns.str.lower()

In [56]:
# check for duplicates
display(df[df.duplicated()])

display(df[df[['name','platform','year_of_release']].duplicated()])


display(df[df[['name','platform','year_of_release']].duplicated(keep='last')])

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
14244,NaN,GEN,1993.0,NaN,0.0,0.00,0.03,0.0,NaN,NaN,NaN
16230,Madden NFL 13,PS3,2012.0,Sports,0.0,0.01,0.00,0.0,83.0,5.5,E


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
604,Madden NFL 13,PS3,2012.0,Sports,2.11,0.22,0.0,0.23,83.0,5.5,E
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.0,0.08,NaN,NaN,NaN


##### <span style="color:green">Observations</span>
Above, we see that there are no duplicates relating to every column, but two games with the same title, platform and release year. It is not worth it to get rid of these records because it is only two and the sales data is useful for us to perform analysis.

In [57]:
# investigate missing values in columns
#name and genre
display(df.query('name.isnull()'))

#checking if there's any games with the name unknown
display(df.query('name == "unkown"'))

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating


##### <span style="color:green">Observations</span>
Above, we see that there are two games with NaN name and genre. They do have the sales figures for each region, however. I believe that the NaN's should be filled with "unkown" so that we can keep the sales data for the Genesis console.

In [58]:
#fill the missing name and genre values with unknown. the other missing values for these rows will be filled later
df[['name','genre']] = df[['name','genre']].fillna(value='unkown') 

In [59]:
#year_of_release
#get the mean of years grouped by platform
mean_years = df.groupby(['platform'])['year_of_release'].transform('mean')

#fillna with these mean values
df['year_of_release'] = df['year_of_release'].fillna(mean_years)

In [60]:
#critic and user scores

# there are duplicate game titles on other platforms. As displayed below
df[df.duplicated(subset=['name'])].sort_values(by='name').head(15)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
3862,Frozen: Olaf's Quest,DS,2013.0,Platform,0.21,0.26,0.00,0.04,NaN,NaN,NaN
14658,007: Quantum of Solace,PC,2008.0,Action,0.01,0.01,0.00,0.00,70.0,6.3,T
1785,007: Quantum of Solace,PS3,2008.0,Action,0.43,0.51,0.02,0.19,65.0,6.6,T
3120,007: Quantum of Solace,Wii,2008.0,Action,0.29,0.28,0.01,0.07,54.0,7.5,T
4475,007: Quantum of Solace,PS2,2008.0,Action,0.17,0.00,0.00,0.26,NaN,NaN,NaN
9507,007: Quantum of Solace,DS,2008.0,Action,0.11,0.01,0.00,0.01,65.0,tbd,T
2243,007: The World is not Enough,PS,2000.0,Action,0.51,0.35,0.00,0.06,61.0,6.7,T
15656,11eyes: CrossOver,PSP,2010.0,Adventure,0.00,0.00,0.02,0.00,NaN,NaN,NaN
10076,18 Wheeler: American Pro Trucker,GC,2002.0,Racing,0.09,0.02,0.00,0.00,52.0,6.8,E
12283,187: Ride or Die,XB,2005.0,Racing,0.05,0.01,0.00,0.00,51.0,6.9,M


In [61]:
#I would like to fill in missing review scores with the mean value from the other platforms if the name of the game is the same
mean_critic_scores = df.groupby('name')['critic_score'].transform('mean')

df['critic_score'] = df['critic_score'].fillna(mean_critic_scores)

##### <span style="color:green">Observations</span>
There are still a lot of missing values in the critic scores, but these should not be assuming by using the mean for the platform. These titles will need to be excluded from visualizations and analysis involving critic_rating

In [62]:
#user_score has values of tbd
df['user_score'].unique()

array(['8', nan, '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4',
       '8.2', '9', '7.9', '8.1', '8.7', '7.1', '3.4', '5.3', '4.8', '3.2',
       '8.9', '6.4', '7.8', '7.5', '2.6', '7.2', '9.2', '7', '7.3', '4.3',
       '7.6', '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '9.4', '6.8',
       '6.1', '6.7', '5.4', '4', '4.9', '4.5', '9.3', '6.2', '4.2', '6',
       '3.7', '4.1', '5.8', '5.6', '5.5', '4.4', '4.6', '5.9', '3.9',
       '3.1', '2.9', '5.2', '3.3', '4.7', '5.1', '3.5', '2.5', '1.9', '3',
       '2.7', '2.2', '2', '9.5', '2.1', '3.6', '2.8', '1.8', '3.8', '0',
       '1.6', '9.6', '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2',
       '2.3', '0.5', '1.3', '0.2', '0.6', '1.4', '0.9', '1', '9.7'],
      dtype=object)

##### <span style="color:green">Observations</span>
I believe that tbd user_score's should be treated as NaN. I would like to fillna with user scores from same game/different platform, but this is not doable if I keep the TBD's which serve no purpose and leave the column as an object datatype.

In [63]:
#replace tbd values with NaN
df['user_score'].replace('tbd',np.nan,inplace=True)

In [64]:
# fix data types

#year_of_release should be changed to int because there should not be any decimals in a year value
df['year_of_release'] = df['year_of_release'].astype(int)

#user_score should be a float because these are quantitative values
df['user_score'] = df['user_score'].astype(float)

In [65]:
#same with the user_scores. doing this after changing it's data type because you cannot get the mean of an object
mean_user_scores = df.groupby('name')['user_score'].transform('mean')

df['user_score'] = df['user_score'].fillna(mean_user_scores)

##### <span style="color:green">Observations</span>

We see now that instead of around 10000 non-null values, we have only 8689, but that's okay because we got rid of unecessary TBD values, filled some of the NaN values, and were able to switch the column to the float data type

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16715 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16715 non-null  int64  
 3   genre            16715 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     9098 non-null   float64
 9   user_score       8689 non-null   float64
 10  rating           9949 non-null   object 
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [67]:
#check if there are games of the same name and different platform that do not all have ratings filled in
display(df[df[['name','platform','rating']].duplicated(subset='name')].sort_values(by='name'))

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
3862,Frozen: Olaf's Quest,DS,2013,Platform,0.21,0.26,0.00,0.04,NaN,NaN,NaN
14658,007: Quantum of Solace,PC,2008,Action,0.01,0.01,0.00,0.00,70.0,6.300,T
1785,007: Quantum of Solace,PS3,2008,Action,0.43,0.51,0.02,0.19,65.0,6.600,T
3120,007: Quantum of Solace,Wii,2008,Action,0.29,0.28,0.01,0.07,54.0,7.500,T
4475,007: Quantum of Solace,PS2,2008,Action,0.17,0.00,0.00,0.26,63.8,6.875,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7149,pro evolution soccer 2011,Wii,2010,Sports,0.07,0.10,0.03,0.02,78.0,5.400,E
4664,pro evolution soccer 2011,PS2,2010,Sports,0.04,0.21,0.05,0.11,77.4,6.700,E
12648,pro evolution soccer 2011,PC,2010,Sports,0.00,0.05,0.00,0.01,79.0,6.120,NaN
15612,uDraw Studio: Instant Artist,X360,2011,Misc,0.01,0.01,0.00,0.00,54.0,5.700,E


In [68]:
#this is the case, so I would like to first fill those NaN ratings with ratings from the same names
#create a dictionary mapping each game name to its rating
missing_rating = df.groupby('name')['rating'].first().to_dict()

#fill in missing rating values using the dictionary
df['rating'] = df.apply(lambda row: missing_rating[row['name']] if pd.isnull(row['rating']) else row['rating'], axis=1)

In [69]:
#run same code as above to check my examples for filled in missing values.
display(df[df[['name','platform','rating']].duplicated(subset='name')].sort_values(by='name'))
df.info()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
3862,Frozen: Olaf's Quest,DS,2013,Platform,0.21,0.26,0.00,0.04,NaN,NaN,None
14658,007: Quantum of Solace,PC,2008,Action,0.01,0.01,0.00,0.00,70.0,6.300,T
1785,007: Quantum of Solace,PS3,2008,Action,0.43,0.51,0.02,0.19,65.0,6.600,T
3120,007: Quantum of Solace,Wii,2008,Action,0.29,0.28,0.01,0.07,54.0,7.500,T
4475,007: Quantum of Solace,PS2,2008,Action,0.17,0.00,0.00,0.26,63.8,6.875,T
...,...,...,...,...,...,...,...,...,...,...,...
7149,pro evolution soccer 2011,Wii,2010,Sports,0.07,0.10,0.03,0.02,78.0,5.400,E
4664,pro evolution soccer 2011,PS2,2010,Sports,0.04,0.21,0.05,0.11,77.4,6.700,E
12648,pro evolution soccer 2011,PC,2010,Sports,0.00,0.05,0.00,0.01,79.0,6.120,E
15612,uDraw Studio: Instant Artist,X360,2011,Misc,0.01,0.01,0.00,0.00,54.0,5.700,E


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16715 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16715 non-null  int64  
 3   genre            16715 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     9098 non-null   float64
 9   user_score       8689 non-null   float64
 10  rating           10386 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [70]:
#checking my theory from the notes that suggested that games were not given ESRB ratings until 
display(df[(df['year_of_release'] < 1994) & (df['rating'].notna())].sample(10,replace=True))

display(df[(df['year_of_release'] > 1994) & (df['rating'].isna())].sample(10,replace=True))

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
279,Teenage Mutant Ninja Turtles,NES,1989,Action,3.38,0.44,0.31,0.04,60.75,8.475000,E
832,NBA Jam,GEN,1992,Sports,1.75,0.25,0.00,0.05,73.25,7.575000,E
2011,Bomberman,NES,1985,Puzzle,0.18,0.00,0.85,0.00,74.00,7.000000,E
14610,Doom,PC,1992,Shooter,0.02,0.00,0.00,0.00,85.00,8.200000,M
279,Teenage Mutant Ninja Turtles,NES,1989,Action,3.38,0.44,0.31,0.04,60.75,8.475000,E
564,Mortal Kombat,GEN,1992,Fighting,1.95,0.63,0.00,0.09,85.00,8.166667,M
14470,Alter Ego,PC,1985,Simulation,0.00,0.03,0.00,0.01,59.00,5.800000,T
861,SimCity,SNES,1991,Simulation,0.93,0.27,0.75,0.04,64.00,2.200000,E10+
14621,SimCity,PC,1988,Simulation,0.00,0.02,0.00,0.01,64.00,2.200000,E10+
14470,Alter Ego,PC,1985,Simulation,0.00,0.03,0.00,0.01,59.00,5.800000,T


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
7253,The Amazing Spider-Man 2 (2014),XOne,2014,Action,0.12,0.08,0.00,0.02,NaN,NaN,None
13711,B.L.U.E.: Legend of Water,PS,1998,Adventure,0.00,0.00,0.04,0.00,NaN,NaN,None
11126,Crayon Shin-Chan Shokkugan! Densetsu o Yobu Om...,DS,2010,Action,0.00,0.00,0.09,0.00,NaN,NaN,None
4452,Jump Start Pet Rescue,Wii,2009,Misc,0.41,0.00,0.00,0.03,NaN,NaN,None
2983,Mercenaries: Playground of Destruction,XB,2005,Action,0.54,0.11,0.00,0.03,NaN,NaN,None
16242,Pet Shop Monogatari DS 2,DS,2010,Simulation,0.00,0.00,0.01,0.00,NaN,NaN,None
9632,Detana TwinBee Yahho! Deluxe Pack,SAT,1995,Shooter,0.00,0.00,0.13,0.00,NaN,NaN,None
3362,New Play Control! Pikmin 2,Wii,2009,Strategy,0.11,0.13,0.33,0.02,NaN,NaN,None
1247,Simpsons Wrestling,PS,2001,Fighting,0.23,1.16,0.00,0.11,NaN,NaN,None
14837,Mojo!,XB,2003,Puzzle,0.02,0.01,0.00,0.00,NaN,NaN,None


##### <span style="color:green">Observations</span>

It looks as though there are plenty of games before 1994 with ratings and tons after 1994 without ratings.

I believe that with about 1/3 of the ratings being NaN, it does not make sense to default a rating based on platform because that will make the analysis inaccurate. I think these NaN's should not be filled and I should perform analysis with what I have. 

Additional Columns

In [71]:
#calculate total sales for each game and add these to a new column
df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales']

#add a column for user_score out of 100 to match the critic_score
df['user_score_100'] = df['user_score'] * 10

In [72]:
#final look at the dataframe before moving on
df.info()
df.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16715 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16715 non-null  int64  
 3   genre            16715 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     9098 non-null   float64
 9   user_score       8689 non-null   float64
 10  rating           10386 non-null  object 
 11  total_sales      16715 non-null  float64
 12  user_score_100   8689 non-null   float64
dtypes: float64(8), int64(1), object(4)
memory usage: 1.7+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales,user_score_100
13500,Ford Bold Moves Street Racing,PSP,2006,Racing,0.04,0.00,0.00,0.00,52.0,NaN,E,0.04,NaN
3242,Beetle Adventure Racing!,N64,1999,Racing,0.40,0.20,0.00,0.02,NaN,NaN,None,0.62,NaN
12598,Fate/Tiger Colosseum Upper,PSP,2008,Fighting,0.00,0.00,0.06,0.00,NaN,NaN,None,0.06,NaN
10033,Yogi Bear: The Video Game,Wii,2010,Action,0.06,0.05,0.00,0.01,NaN,NaN,E,0.12,NaN
7617,Tekken 3D: Prime Edition,3DS,2012,Fighting,0.06,0.07,0.05,0.01,64.0,5.9,T,0.19,59.0
6935,NASCAR 08,PS2,2007,Racing,0.12,0.09,0.00,0.03,56.0,8.2,E,0.24,82.0
4609,MLB 08: The Show,PS2,2008,Sports,0.35,0.01,0.00,0.06,84.0,7.3,E,0.42,73.0
1551,Kid Icarus: Uprising,3DS,2012,Action,0.48,0.35,0.36,0.07,83.0,8.7,E10+,1.26,87.0
1638,Intelligent Qube,PS,1997,Puzzle,0.13,0.07,1.00,0.02,NaN,NaN,None,1.22,NaN
9074,PaRappa The Rapper 2,PS2,2001,Misc,0.07,0.05,0.00,0.02,67.0,6.6,E,0.14,66.0


### Exploratory Data Analysis

#### Analyze the transformed data with the following questions

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
1. Look at how many games were released in different years. Is the data for every period significant?

In [73]:
#grab the unique years and how many games were released in each year
unique_games_by_year = df.groupby('year_of_release')['name'].count().reset_index()

In [74]:
#plot a bar chart showing the distribution of unique games released by year

#create custom labels
custom_labels_y = {
     'name':'Number of Games', # Rename 'name' column to 'Number of Games'
     'year_of_release':'Year' # Rename 'year_of_release' column to 'Year Released'
}

#create bar chart
fig20 = px.bar(unique_games_by_year,
              x='year_of_release',
              y='name',
              labels=custom_labels_y,
              title='<b> Unique Games Released by Year <b>',
              template='plotly_dark')

fig20.update_layout(height=1000)
fig20.show()

#####  <span style="color:green">Observations</span>
Based on this result, it looks like not many games were release in the 90s and before, specifically before 1994.

Game development ramped up starting in 2002 and there is a lot of data following that year. I believe that the data drom 1994 to 2016 is significant, but the most significant period is from 2006-2016

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
2. Look at how sales varied from platform to platform. Choose the platforms with the greatest total sales and build a distribution based on data for each year. Find platforms that used to be popular but now have zero sales. How long does it generally take for new platforms to appear and old ones to fade?

In [75]:
#top overall sales by platform
sales_by_platform = df.groupby('platform')['total_sales'].sum().sort_values(ascending=False).reset_index()

In [76]:
#plot a bar chart showing the distribution of total sales by platform

#create custom labels
custom_labels_z = {
     'platform':'Platform', 
     'total_sales':'Total Sales' 
}

#create bar chart
fig21 = px.bar(sales_by_platform,
              x='platform',
              y='total_sales',
              labels=custom_labels_z,
              title='<b> Total Sales by Platform <b>',
              template='plotly_dark')

fig21.update_layout(height=1000)
fig21.update_xaxes(tickangle=45)
fig21.show()

##### <span style="color:green">Plan</span>

From the above list, I will grab the consoles that made over 300 million USD in their lifetime.

I will then plot a bar graph to show the sales by year and color the bars by the consoles to show who was selling the most

In [77]:
#group the dataframe by platform and aggregate by total_sales.
df_total_sales = df.groupby(['platform'])['total_sales'].sum().reset_index()

#grab the consoles that have made over 300 million into a list
df_over_300 = df_total_sales.query('total_sales > 300')

#create a new dataframe with the platforms that have over 300 million USD in sales
df_grouped = df[df['platform'].isin(df_over_300['platform'])].groupby(['platform','year_of_release'])['total_sales'].sum().reset_index()

In [78]:
#plot a bar chart showing the distribution of sales by year and by platform

#create custom labels
custom_labels = {
     'platform':'Platform', # Rename 'platform' column to 'Platform'
     'total_sales':'Sales (in USD million)',  # Rename 'sales' column to 'Sales'
     'year_of_release':'Year' # Rename 'year_of_release' column to 'Year Released'
}

#create bar chart
fig1 = px.bar(df_grouped,
              x='year_of_release',
              y='total_sales',
              color='platform',
              labels=custom_labels,
              title='<b> Total Sales by Year for Most Popular Platforms <b>',
              template='plotly_dark')
fig1.update_xaxes(range=[1993, 2017])
fig1.update_layout(height=1000)
fig1.show()

#####  <span style="color:green">Observations</span>

Based on the above graph, we can see the most popular consoles since 1994.

At first glance, we can see that the PS2 is one of the most popular of all time as of 2016 and the Xbox 360, Nintendo Wii and Playstation 3 compare to it.

In 2004, the PS2 had around 234 million in game sales. It's sales only dropped off after the release of the PS3 in 2006, which is expected

In [79]:
#to find platforms that used to be popular and then dropped off in sales, I will broaden my search to grab consoles that made at least 60 million in one year in game sales

#dataframe of all consoles sales by year
df_sales_by_year = df.groupby(['platform','year_of_release'])['total_sales'].sum().reset_index()

#grab the consoles that have made over 60 million into a list
df_over_60 = df_sales_by_year.query('total_sales > 60')
df_over_60_unique = df_over_60['platform'].unique()

#create a new dataframe with the platforms that had over 60 million USD in sales in one year
df_grouped_x = df[df['platform'].isin(df_over_60_unique)].groupby(['platform','year_of_release'])['total_sales'].sum().reset_index()

In [80]:
#plot a bar chart showing the distribution of sales by year and by platform

#create bar chart
fig2 = px.bar(df_grouped_x,
              x='year_of_release',
              y='total_sales',
              color='platform',
              labels=custom_labels,
              title='<b> Total Sales by Year for Popular Platforms <b>',
              template='plotly_dark')

fig2.update_xaxes(range=[1987, 2017])              
fig2.update_layout(height=1000)
fig2.show()

In [81]:
#from the dataset, I would like to pull how long these consoles stay popular for. I'm judging this by consoles that aren't new (released within the last 4 years)
# and by how many years they made about 60 million in game sales.
df_years_popular = df_grouped_x.query('platform not in ("XOne","PS4") and total_sales > 60').groupby('platform')['year_of_release'].count()

display(df_years_popular)

#get the mean for all the platforms and print it out
print('The average time a big platform stays popular:',df_years_popular.mean(), 'years')

platform
3DS     1
DS      6
GB      1
GBA     4
PS      5
PS2     7
PS3     7
Wii     5
X360    7
XB      2
Name: year_of_release, dtype: int64

The average time a big platform stays popular: 4.5 years


#####  <span style="color:green">Observations</span>

Based on the bar graph, popular platforms generally have life spans of around 10 years, but they're only very popular for about half of that time.

The average years a platform has a lot of game sales is 4.5 years. Some platforms like the 3DS and Gameboy only had one year of great success, while giants like the PS2,PS3 and Xbox 360 stayed popular for 7 whole years.

This is displayed in the distribution where around every 4-5 years, new platforms begin to rise and old ones begin to die out.

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
3. Determine what period you should take data for. To do so, look at your answers to the previous questions. The data should allow you to build a model for 2017.

 
 Work only with the data that you've decided is relevant. Disregard the data for previous years.

Since we are making a model for 2017 sales, it is important that we use sales from consoles that are current active and thriving to this day.

These include the PS4, Xbox One, and 3DS. To fit this criteria, our data should start the year the PS4 and Xbox One were released (2013).

The data period for this model will be from 2013-2016

In [82]:
df_2013_2016 = df.query('2013 <= year_of_release <= 2017')

df_total_sales_2013_2016 = df_2013_2016.groupby(['platform'])['total_sales'].sum().sort_values(ascending=False).reset_index()

df_2016 = df.query('year_of_release == 2016')

df_total_sales_2016 = df_2016.groupby(['platform'])['total_sales'].sum().sort_values(ascending=False).reset_index()

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
4. Which platforms are leading in sales? Which ones are growing or shrinking? Select several potentially profitable platforms.

In [83]:
#create bar chart to visualize total sales in the time period 2013-2016
custom_labels_new = {
     'platform':'Platform', # Rename 'platform' column to 'Platform'
     'total_sales':'Sales (in USD million)'  # Rename 'sales' column to 'Sales'
}
fig3 = px.bar(df_total_sales_2013_2016,
              x='platform',
              y='total_sales',
              labels=custom_labels_new,
              title='<b> Total Sales by Platform between 2013 and 2016 <b>',
              template='plotly_dark')
              
fig3.update_layout(height=1000)
fig3.show()

In [84]:
#create bar chart to visualize total sales in 2016 to show if the sales above keep up with the current year
fig4 = px.bar(df_total_sales_2016,
              x='platform',
              y='total_sales',
              labels=custom_labels_new,
              title='<b> Total Sales by Platform in 2016 <b>',
              template='plotly_dark')
              
fig4.update_layout(height=500)
fig4.show()

#####  <span style="color:green">Observations</span>

Based on the two bar graphs displaying total sales for 2013-2016 and 2016 alone, the PS4 and Xbox One have consistently high sales. This makes sense as they are newer platforms. The 3DS is declining in sales as it is aging, but I think it will be useful to include in our data as it is has the highest sales amongst current Nintendo Platforms. The Xbox 360 is not relevant for this analysis because it's sales have plummeted in the past few years and it has effectively been replaced by the Xbox One.

I will filter the dataframe again only keeping the PS4, Xbox One, and 3DS.

In [85]:
df_2013_2016_current = df_2013_2016.query('platform in ("PS4","XOne","3DS")')

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
5. Build a box plot for the global sales of all games, broken down by platform. Are the differences in sales significant? What about average sales on various platforms? Describe your findings.

In [86]:
#get the average sales for each platform
display(df_2013_2016_current.groupby('platform')['total_sales'].mean())
#get the amount of games for each platform
display(df_2013_2016_current.groupby('platform')['total_sales'].count())

platform
3DS     0.464437
PS4     0.801378
XOne    0.645020
Name: total_sales, dtype: float64

platform
3DS     311
PS4     392
XOne    247
Name: total_sales, dtype: int64

In [87]:
#create a box plot showing the total sales statistics by platform. You can view the specific statistics numbers by hovering over the boxplots.
fig5 = px.box(df_2013_2016_current,
       x='platform',
       y='total_sales',
       labels=custom_labels_new,
       title='<b> Sales by Platform between 2013 and 2016 <b>',
       template='plotly_dark')

fig5.update_layout(height=900)
fig5.show()

In [91]:
#boxplot without the outliers
fig22 = px.box(df_2013_2016_current,
       x='platform',
       y='total_sales',
       labels=custom_labels_new,
       title='<b> Sales by Platform between 2013 and 2016 (Outliers Excluded) <b>',
       template='plotly_dark')

fig22.update_layout(height=900)
fig22.update_yaxes(range=[0, 2.5])   
fig22.show()

In [35]:
#get games with total_sales over 4 million dollars in our time period
df_2013_2016_current.query('total_sales > 4')

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales,user_score_100
31,Call of Duty: Black Ops 3,PS4,2015,Shooter,6.03,5.86,0.36,2.38,NaN,NaN,None,14.63,NaN
33,Pokemon X/Pokemon Y,3DS,2013,Role-Playing,5.28,4.19,4.35,0.78,NaN,NaN,None,14.60,NaN
42,Grand Theft Auto V,PS4,2014,Action,3.96,6.31,0.38,1.97,97.0,8.3,M,12.62,83.0
47,Pokemon Omega Ruby/Pokemon Alpha Sapphire,3DS,2014,Role-Playing,4.35,3.49,3.10,0.74,NaN,NaN,None,11.68,NaN
77,FIFA 16,PS4,2015,Sports,1.12,6.12,0.06,1.28,82.0,4.3,E,8.58,43.0
87,Star Wars Battlefront (2015),PS4,2015,Shooter,2.99,3.49,0.22,1.28,NaN,NaN,None,7.98,NaN
92,Call of Duty: Advanced Warfare,PS4,2014,Shooter,2.81,3.48,0.14,1.23,83.0,5.7,M,7.66,57.0
94,FIFA 17,PS4,2016,Sports,0.66,5.75,0.08,1.11,85.0,5.0,E,7.60,50.0
96,Super Smash Bros. for Wii U and 3DS,3DS,2014,Fighting,3.27,1.37,2.43,0.48,NaN,NaN,None,7.55,NaN
99,Call of Duty: Black Ops 3,XOne,2015,Shooter,4.59,2.11,0.01,0.68,NaN,NaN,None,7.39,NaN


####  <span style="color:green">Observations</span>

Based on the boxplots above, there are a ton of games released by each system that do not sell well. This drives down the medians to values below 250,000 USD for every platform and 75 percentile's to under $750,000 for each platform

##### <span style="color:skyblue">Outliers</span>
All systems have many outliers which heavily boosts each systems annual sales. They seem to rely on big ticket games such as Call of Duty, Grand Theft Auto, FIFA, and Pokemon.

##### <span style="color:skyblue">Significant Differences in Sales</span>
The differences in game sales are not too significant, but the 3DS has less big ticket games as outliers to carry their sales numbers. The Xbox One has most of the same games the Playstation 4 has, but PS4 has a bigger game library.

##### <span style="color:skyblue">Average Sales</span>
The Playstation 4 has higher average sales than the other two consoles by a decent amount. The average game on PS4 makes about $800,000, compared to $640,000 on the Xbox One. The 3DS lags behind with average sales of around $460,000 per game in this time period, heavily relying on Pokemon games to boost sales. Although, the Xbox One has a higher median sales total of $220,000 compared to PS4's $200,000

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
6. Take a look at how user and professional reviews affect sales for one popular platform (you choose). Build a scatter plot and calculate the correlation between reviews and sales. Draw conclusions.

In [36]:
#PS4
df_2013_2016_current_PS4 = df_2013_2016_current.query('platform == "PS4"')

In [37]:
#plot a scatter showing the distribution between critic_score and total_sales
fig6 = px.scatter(df_2013_2016_current_PS4,
                  x='critic_score',
                  y='total_sales',
                  labels={'critic_score':'Critic Score','total_sales':'Total Sales','name':'Game'},
                  title='<b> Sales by Critic Score for PS4 between 2013 and 2016 <b>',
                  color_discrete_sequence=['skyblue'],
                  hover_data=['name'],
                  template='plotly_dark')

               
fig6.update_layout(height=900)
fig6.show()

In [38]:
#plot a scatter showing the distribution between user_score and total_sales
fig7 = px.scatter(df_2013_2016_current_PS4,
                  x='user_score_100',
                  y='total_sales',
                  labels={'user_score_100':'User Score','total_sales':'Total Sales','name':'Game'},
                  title='<b> Sales by User Score for PS4 between 2013 and 2016 <b>',
                  color_discrete_sequence=['green'],
                  hover_data=['name'],
                  template='plotly_dark')
fig7.update_layout(height=900)
fig7.show()

In [39]:
#calculate the correlation coefficient's for each
print('The correlation coefficient for user_score and total_sales is:',df_2013_2016_current_PS4['user_score_100'].corr(df_2013_2016_current_PS4['total_sales']))

print('The correlation coefficient for critic_score and total_sales is:',df_2013_2016_current_PS4['critic_score'].corr(df_2013_2016_current_PS4['total_sales']))

The correlation coefficient for user_score and total_sales is: -0.02161249320047854
The correlation coefficient for critic_score and total_sales is: 0.39155120045082015


####  <span style="color:green">Observations</span>

Based on the plots and the correlation calculations, there is a moderately positive linear relationship between critic_score and total_sales.
 - This means that lower critic scores somewhat affect whether users will purchase a game or not and are a bit more likely to buy a game that has a high critic rating.

On the other hand, there is no association between user_scores and total_sales. 
 - This means that users tend to not care about how other users rate games.

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
7. Keeping your conclusions in mind, compare the sales of the same games on other platforms.

In [40]:
#create a dataframe that contains games from PS4 that are provided on the Xbox One or 3DS as well
df_2013_2016_current_duplicated = df_2013_2016_current[df_2013_2016_current.duplicated(subset='name',keep=False)]

#filter out the PS4 versions so we can create visualizations on just the games from other platforms
df_2013_2016_current_duplicated_no_PS4 = df_2013_2016_current_duplicated.query('platform not in "PS4"')

In [41]:
#plot a scatter showing the distribution between critic_score and total_sales
fig7 = px.scatter(df_2013_2016_current_duplicated_no_PS4,
                  x='critic_score',
                  y='total_sales',
                  labels={'critic_score':'Critic Score','total_sales':'Total Sales','name':'Game','platform':'Platform'},
                  title='<b> Sales by Critic Score for Xbox One and 3DS between 2013 and 2016 <b>',
                  color_discrete_sequence=['skyblue'],
                  hover_data=['name','platform'],
                  template='plotly_dark')

               
fig7.update_layout(height=900)
fig7.show()

In [42]:
#plot a scatter showing the distribution between user_score and total_sales
fig8 = px.scatter(df_2013_2016_current_duplicated_no_PS4,
                  x='user_score_100',
                  y='total_sales',
                  labels={'user_score_100':'User Score','total_sales':'Total Sales','name':'Game','platform':'Platform'},
                  title='<b> Sales by Critic Score for Xbox One and 3DS between 2013 and 2016 <b>',
                  color_discrete_sequence=['green'],
                  hover_data=['name','platform'],
                  template='plotly_dark')

               
fig8.update_layout(height=900)
fig8.show()

In [43]:
#calculate the correlation coefficient's for each
print('The correlation coefficient for user_score and total_sales is:',df_2013_2016_current_duplicated_no_PS4['user_score_100'].corr(df_2013_2016_current_duplicated_no_PS4['total_sales']))

print('The correlation coefficient for critic_score and total_sales is:',df_2013_2016_current_duplicated_no_PS4['critic_score'].corr(df_2013_2016_current_duplicated_no_PS4['total_sales']))

The correlation coefficient for user_score and total_sales is: -0.12105622884464032
The correlation coefficient for critic_score and total_sales is: 0.34430036946561626


####  <span style="color:green">Observations</span>

Based on the plots and the correlation coefficients for the games that are on both PS4 and Xbox One/3DS, there is a weaker negative linear relationship between user_score and total_sales and still a positive correlation between critic_score and total_sales

It looks to be universally consistent that critic scores matter more to gamers than user scores.

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
8. Take a look at the general distribution of games by genre. What can we say about the most profitable genres? Can you generalize about genres with high and low sales?

In [44]:
#create a series of counts of games by genre to plot on a bar graph
df_2013_2016_current_genre = df_2013_2016_current.groupby('genre')['name'].count().sort_values(ascending=False).reset_index()
df_2013_2016_current_genre

,genre,name
0,Action,346
1,Role-Playing,128
2,Sports,93
3,Shooter,81
4,Misc,71
5,Adventure,70
6,Racing,40
7,Fighting,32
8,Platform,32
9,Simulation,27


In [45]:
#create a bar graph of the total amount of games by genre in our time period

fig9 = px.bar(df_2013_2016_current_genre,
              x='genre',
              y='name',
              labels={'genre':'Genre','name':'Number of Games'},
              title='<b> Number of Games by Genre from 2013 to 2016 <b>',
              color_discrete_sequence=px.colors.qualitative.G10,
              template='plotly_dark')
              
fig9.update_layout(height=700)

fig9.show()

In [102]:
#check the average sales of each genre
avg_sales_by_genre = df_2013_2016_current.groupby('genre')['total_sales'].mean().sort_values(ascending=False).reset_index()

In [103]:
#create a bar graph of the average sales by genre

fig23 = px.bar(avg_sales_by_genre,
              x='genre',
              y='total_sales',
              labels={'genre':'Genre','total_sales':'Average Sales (USD Millions)'},
              title='<b> Average Sales by Genre <b>',
              color_discrete_sequence=px.colors.qualitative.G10,
              template='plotly_dark')
              
fig23.update_layout(height=700)

fig23.show()

In [107]:
#create a dataframe containing the games from the genres with the highest average sales
df_2013_2016_current_best_genre = df_2013_2016_current.query('genre in ("Action","Fighting","Racing","Platform","Role-Playing","Sports","Shooter")')

In [110]:
#box plot showing distribution between genre and sales

fig24 = px.box(df_2013_2016_current_best_genre,
              x='genre',
              y='total_sales',
              labels={'total_sales':'Total Sales (USD Millions)','genre':'Genre'},
              title='<b> Sales by Genre <b>',
              hover_data=['genre'],
              color_discrete_sequence=px.colors.qualitative.Dark2,
              template='plotly_dark')
              
fig24.update_layout(height=700)
fig24.update_yaxes(range=[0,7])
fig24.show()

####  <span style="color:green">Observations</span>

According to the bar graphs above, the majority of games released in the years 2013-2016 were Action games. 

When you take Average Sales of each game into account, we see genres like Fighting, Racing, and Platform outperform Action games on average. The Action genre produces more titles, but Fighting, Racing, and Platform outperform those titles on average.

In the boxplot, we can see that the Action genre is carried by many outliers while keeping a low median of 125,000 USD per title. Shooters don't get many releases, but dominate the sales.

All these genres will be needed.

### Create User Profile for each Region

<div style="border-radius: 15px; border: 3px solid skyblue; padding: 15px;">

For each region (NA, EU, JP), determine:

- The top five platforms. Describe variations in their market shares from region to region.
- The top five genres. Explain the difference.
- Do ESRB ratings affect sales in individual regions?

#### North America

In [48]:
#sales of the top 5 selling platforms in the time period
df_2013_2016_platform_sales_na = df_2013_2016.groupby('platform')['na_sales'].sum().sort_values(ascending=False).head(5).reset_index()

#sales of the top 5 selling genres in the time period
df_2013_2016_genre_sales_na = df_2013_2016.groupby('genre')['na_sales'].sum().sort_values(ascending=False).head(5).reset_index()

In [49]:
#plot a bar graph showing the distribution between platform and sales
fig14 = px.bar(df_2013_2016_platform_sales_na,
                  x='platform',
                  y='na_sales',
                  labels={'rating':'ESRB Rating','platform':'Platform','na_sales':'Sales'},
                  title='<b> Sales by Platform in North America <b>',
                  category_orders={'rating':['E','E10+','T','M']},
                  color_discrete_sequence= px.colors.qualitative.T10,
                  template='plotly_dark')

               
fig14.update_layout(height=900)
fig14.show()

In [50]:
#plot a bar graph showing the distribution between genre and sales
fig15 = px.bar(df_2013_2016_genre_sales_na,
                  x='genre',
                  y='na_sales',
                  labels={'rating':'ESRB Rating','genre':'Genre','na_sales':'Sales'},
                  title='<b> Sales by Genre in North America <b>',
                  color_discrete_sequence= px.colors.qualitative.T10_r,
                  template='plotly_dark')

               
fig15.update_layout(height=900)
fig15.show()

####  <span style="color:green">Observations</span>

The PS4 looks to be the most popular console in North America in the time period by a narrow margin over the Xbox One. The Xbox 360 is close behind, but the console was at the end of it's lifespan

Action and Shooter games are the clear leaders in genres in North America.

In [51]:
#plot a bar graph showing the distribution between ESRB ratings and sales
fig11 = px.bar(df_2013_2016.sort_values(by='na_sales'),
                  x='rating',
                  y='na_sales',
                  labels={'rating':'ESRB Rating','total_sales':'Total Sales','name':'Game','platform':'Platform','na_sales':'Sales'},
                  title='<b> Sales by ESRB Rating in North America <b>',
                  color_discrete_sequence= px.colors.sequential.Plasma_r,
                  hover_data=['name','platform','rating'],
                  template='plotly_dark')

               
fig11.update_layout(height=900)
fig11.show()

####  <span style="color:green">Observations</span>

Based on the bar graph above, M rated games have the most sales amongst the ESRB ratings in North America. E, E 10+ and T rated games are fairly close to each other, but T rated games appear to lag behind the rest.

Seeing the Action and Shooter games are the most popular in North America, those types of games are more likely to be rated M.

#### Europe

In [52]:
#sales of the top 5 selling platforms in the time period
df_2013_2016_platform_sales_eu = df_2013_2016.groupby('platform')['eu_sales'].sum().sort_values(ascending=False).head(5).reset_index()

#sales of the top 5 selling genres in the time period
df_2013_2016_genre_sales_eu = df_2013_2016.groupby('genre')['eu_sales'].sum().sort_values(ascending=False).head(5).reset_index()

In [53]:
#plot a bar graph showing the distribution between platform and sales
fig16 = px.bar(df_2013_2016_platform_sales_eu,
                  x='platform',
                  y='eu_sales',
                  labels={'rating':'ESRB Rating','platform':'Platform','na_sales':'Sales'},
                  title='<b> Sales by Platform in Europe <b>',
                  color_discrete_sequence= px.colors.qualitative.T10,
                  template='plotly_dark')

               
fig16.update_layout(height=900)
fig16.show()

In [54]:
#plot a bar graph showing the distribution between genre and sales
fig17 = px.bar(df_2013_2016_genre_sales_eu,
                  x='genre',
                  y='eu_sales',
                  labels={'rating':'ESRB Rating','genre':'Genre','na_sales':'Sales'},
                  title='<b> Sales by Genre in Europe <b>',
                  color_discrete_sequence= px.colors.qualitative.T10_r,
                  template='plotly_dark')

               
fig17.update_layout(height=900)
fig17.show()

####  <span style="color:green">Observations</span>

In Europe, The Playstation 4 is still the most popular platform, but this time by a wide margin. The current generation console Xbox One even lags behind the previous generation PS3.

The top 4 genres in Europe are identical to North America with Action games being the clear front runner in the region.

In [55]:
#plot a bar graph showing the distribution between ESRB ratings and sales
fig12 = px.bar(df_2013_2016.sort_values(by='eu_sales'),
                  x='rating',
                  y='eu_sales',
                  labels={'rating':'ESRB Rating','total_sales':'Total Sales','name':'Game','platform':'Platform','na_sales':'Sales'},
                  title='<b> Sales by ESRB Rating in Europe <b>',
                  color_discrete_sequence= px.colors.qualitative.D3_r,
                  hover_data=['name','platform','rating'],
                  template='plotly_dark')

               
fig12.update_layout(height=900)
fig12.show()

####  <span style="color:green">Observations</span>

This distribution is almost identical to North America's. It appears people like M rated games more than others. E rated games include sports games as you can see in the bar graph when hovering over the sub bars. This would explain their popularity over E10+ and T rated games.

#### Japan

In [56]:
#display the sales of the top 5 selling platforms in the time period
df_2013_2016_platform_sales_jp = df_2013_2016.groupby('platform')['jp_sales'].sum().sort_values(ascending=False).head(5).reset_index()

#display the sales of the top 5 selling genres in the time period
df_2013_2016_genre_sales_jp = df_2013_2016.groupby('genre')['jp_sales'].sum().sort_values(ascending=False).head(5).reset_index()

In [57]:
#plot a bar graph showing the distribution between platform and sales
fig18 = px.bar(df_2013_2016_platform_sales_jp,
                  x='platform',
                  y='jp_sales',
                  labels={'rating':'ESRB Rating','platform':'Platform','na_sales':'Sales'},
                  title='<b> Sales by Platform in Japan <b>',
                  color_discrete_sequence= px.colors.qualitative.T10,
                  template='plotly_dark')

               
fig18.update_layout(height=900)
fig18.show()

In [58]:
#plot a bar graph showing the distribution between genre and sales
fig19 = px.bar(df_2013_2016_genre_sales_jp,
                  x='genre',
                  y='jp_sales',
                  labels={'rating':'ESRB Rating','genre':'Genre','na_sales':'Sales'},
                  title='<b> Sales by Genre in Japan <b>',
                  color_discrete_sequence= px.colors.qualitative.T10_r,
                  template='plotly_dark')

               
fig19.update_layout(height=900)
fig19.show()

####  <span style="color:green">Observations</span>

Japan's numbers differ from EU and NA. The 3DS is by far the most popular platform in the region and you see more handheld consoles being popular such as the PSV.
The PS4 hasn't quite sold much in the region as it's still outsold in the time period by its predecessor, the PS3.

The most popular genre differs from the other two regions as well. Role-playing games are the region wide leader with Action games following close behind. Other genres are not super popular in Japan

In [59]:
#plot a bar graph showing the distribution between ESRB ratings and sales
fig13 = px.bar(df_2013_2016.sort_values(by='jp_sales'),
                  x='rating',
                  y='jp_sales',
                  labels={'rating':'ESRB Rating','total_sales':'Total Sales','name':'Game','platform':'Platform','na_sales':'Sales'},
                  title='<b> Sales by ESRB Rating in Japan <b>',
                  category_orders={'rating':['E','E10+','T','M']},
                  color_discrete_sequence= px.colors.qualitative.Set3,
                  hover_data=['name','platform','rating'],
                  template='plotly_dark')

fig13.update_yaxes(range=[0, 25])               
fig13.update_layout(height=900)
fig13.show()

####  <span style="color:green">Observations</span>

The distribution of the above bar graph differs greatly from NA and EU. T rated games are the most popular in this region as they were the least popular in the other 2 regions.

E rated games and M rated games have around the same sales with E10+ rated games being the least popular as it is in other regions. 

####  <span style="color:skyblue">Overall Observations</span>

- North American and European gamers appear to be on the same page when it comes to what games to play. Action, Shooters, and Sports games dominate the regions and should have emphasis placed on them.

- Japanese gamers have different general preferences. They prefer Role-Playing games with some Action games as well. They also prefer Japanese consoles such as Nintendo and Playstation while NA and EU gamers prefer Playstation and Xbox. We will want to market Nintendo games more than other platforms to Japanese users.

- In terms of ESRB ratings, Japanese gamers tend to prefer Teen rated games considering Nintendo consoles do not offer many M rated games. NA and EU users heavily prefer M rated games such as Call of Duty and Grand Theft Auto.

### Statistical Data Analysis

<div style="border-radius: 15px; border: 3px solid skyblue; padding: 15px;">

Test the following hypotheses:

- Average user ratings of the Xbox One and PC platforms are the same. 
- Average user ratings for the Action and Sports genres are different.
    - The Null hypotheses for both of these test would be that the user ratings are not the same since we are testing for the difference.


I'll be using a significance level of .05 for these tests. 
I'm doing this because we have a rather small sample size. I will test different significance levels depending on results.

In [60]:
#create the dataframes for these tests
df_2013_2016_xbox_one = df_2013_2016.query('platform == "XOne"')
df_2013_2016_pc = df_2013_2016.query('platform == "PC"')

df_2013_2016_action = df_2013_2016.query('genre == "Action"')
df_2013_2016_sports = df_2013_2016.query('genre == "Sports"')

In [61]:
#show the statistics for each user_score
print('First Hypothesis:')
print('Xbox One')
print('User Score Variance:',df_2013_2016_xbox_one['user_score'].var())
print('User Score Mean:',df_2013_2016_xbox_one['user_score'].mean())
print('User Score Standard Deviation:',df_2013_2016_xbox_one['user_score'].std())
print()
print('PC')
print('PC User Score Variance:',df_2013_2016_pc['user_score'].var())
print('PC User Score Mean:',df_2013_2016_pc['user_score'].mean())
print('PC User Score Standard Deviation:',df_2013_2016_pc['user_score'].std())
print()
print('Second Hypothesis:')
print('Action')
print('Action Genre User Score Variance:',df_2013_2016_action['user_score'].var())
print('Action Genre User Score Mean:',df_2013_2016_action['user_score'].mean())
print('Action Genre User Score Standard Deviation:',df_2013_2016_action['user_score'].std())
print()
print('Sports')
print('Sports Genre User Score Variance:',df_2013_2016_sports['user_score'].var())
print('Sports Genre User Score Mean:',df_2013_2016_sports['user_score'].mean())
print('Sports Genre User Score Standard Deviation:',df_2013_2016_sports['user_score'].std())

First Hypothesis:
Xbox One
User Score Variance: 1.8992091736390664
User Score Mean: 6.604085760517799
User Score Standard Deviation: 1.3781179824815677

PC
PC User Score Variance: 3.053022200446405
PC User Score Mean: 6.252781316348195
PC User Score Standard Deviation: 1.7472899588924573

Second Hypothesis:
Action
Action Genre User Score Variance: 1.788163252158323
Action Genre User Score Mean: 6.859161208151383
Action Genre User Score Standard Deviation: 1.3372222149509494

Sports
Sports Genre User Score Variance: 3.015458270290192
Sports Genre User Score Mean: 5.191597222222222
Sports Genre User Score Standard Deviation: 1.7365074921491679


####  <span style="color:green">Observations</span>

The variances for the samples of both hypotheses are not equal, therefore we will have equal_var set for False for each test

In [62]:
'''
Null Hypothesis: Average user ratings of the Xbox One and PC platforms are different.
Alternate Hypothesis: Average user ratings of the Xbox One and PC platforms are the same.
'''
alpha = .05

#using ttest_ind() because we are testing that the means of the two statistical populations are equal based on samples taken from them
results = stats.ttest_ind(df_2013_2016_xbox_one['user_score'],df_2013_2016_pc['user_score'],equal_var=False,nan_policy='omit')

print(results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis")
else:
    print("We can't reject the null hypothesis")


0.038875966076512496
We reject the null hypothesis


####  <span style="color:green">Observations</span>

We can conclude above that we should reject the null hypothesis that average user ratings of the Xbox One and PC platforms are different. The means in our statistics print statements are similar so this is a valid conclusion

In [63]:
'''
Null Hypothesis: Average user ratings for the Action and Sports genres are the same.
Alternate Hypothesis: Average user ratings for the Action and Sports genres are different.
'''

#using ttest_ind() because we have independent samples from two groups and want to test whether there is a significant difference between their means.
results = stats.ttest_ind(df_2013_2016_action['user_score'],df_2013_2016_sports['user_score'],equal_var=False,nan_policy='omit')

print(results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis")
else:
    print("We can't reject the null hypothesis")

1.8241167772578782e-25
We reject the null hypothesis


####  <span style="color:green">Observations</span>

We can conclude that the null hypothesis of average user ratings for the Action and Sports genres are the same should be rejected. We accept the alternate hypothesis

## Conclusion

<div style="border-radius: 15px; border: 3px solid brown; padding: 15px;">

 - From the exploratory data analysis, I determined which period of time was most significant for this model. We want to formulate a plan for 2017 and must use game data from platforms that are currently on the market and thriving.

 - I looked at the platforms that had the highest game sales over the years and found that the PS2 has the highest sales of all time.

 - I found that the average lifespan of a popular gaming console is around 10 year, with about 4.5 of those years being it's peak in sales.

 - In order to perform analysis for our 2017 model, I needed to get the best performing current consoles which I determined to be the PS4, Xbox One, and 3DS.

 - I analyzed the sales of these platforms, measured whether reviews had impacts on sales, and measured which genres generated the most revenue. I can concludethat the PS4 is the most popular console on the market currently, with Action games leading the way in sales and users tend to respond well to high Critic Scores rather than reviews from other users when buying games.

 - North America sells the most games and prefers the PS4 and Xbox One for their platforms.

 - I concluded at the end that the average user ratings for the Action and Sports genres are different and the average user ratings of the Xbox One and PC platforms are the same.